## Generate data from sims.

In [1]:
from shared.preface import *
import shared.functions as fct

# Initialize parameters and files.
PRE = PRE(
    # sim='L006N188', 
    sim='L012N376', 
    z0_snap=62, z4_snap=13, DM_lim=1000,
    sim_dir=SIM_ROOT, sim_ver=SIM_TYPE,
    phis=10, thetas=10, vels=100,
    pre_CPUs=6, sim_CPUs=6
)

mass_gauge = 12.3
mass_range = 0.3
size = 1
hname = f'1e+{mass_gauge}_pm{mass_range}Msun'
fct.halo_batch_indices(
    PRE.Z0_STR, mass_gauge, mass_range, 'halos', size, 
    hname, PRE.SIM_DIR, PRE.OUT_DIR
)
halo_batch_IDs = np.load(f'{PRE.OUT_DIR}/halo_batch_{hname}_indices.npy')
halo_batch_params = np.load(f'{PRE.OUT_DIR}/halo_batch_{hname}_params.npy')
halo_num = len(halo_batch_params)

print('********Number density band********')
print('Halo batch params (Rvir,Mvir,cNFW):')
print(halo_batch_params)
print('***********************************')


mass_ticks = 100
mass_range_eV = np.geomspace(0.01, 0.3, mass_ticks)*eV  # 10 to 300 meV

etas_arr = []
for halo_j in range(halo_num):

    # Load velocities.
    tot_name = f'{hname}_halo{halo_j}'
    fname = f'{PRE.NUS}nus_{tot_name}'
    u_all = fct.load_sim_data(PRE.OUT_DIR, fname, 'velocities')

    # Calculate number density.
    out_file = f'{PRE.OUT_DIR}/number_densities_{PRE.NUS}nus_{tot_name}.npy'
    fct.number_densities_mass_range(
        u_all, mass_range_eV, out_file,
        average=False, m_start=0.01, z_start=0.5
    )

    etas = np.load(f'{out_file}')/N0
    etas_arr.append(etas)

etas_sim = np.array(etas_arr).reshape(halo_num, mass_ticks)


# Load data for MW halo only of smooth simulation
nus_smooth = 10000
MW_HALO = True
VC_HALO = False
AG_HALO = False
halos = 'MW'*MW_HALO + '+VC'*VC_HALO + '+AG'*AG_HALO
sim = 'LinfNinf'
smooth_name = f'nus_{nus_smooth}_halos_{halos}_{SOLVER}'
u_all_MW = fct.load_sim_data(sim, smooth_name, 'velocities')

output = f'{sim}/number_densities_{smooth_name}.npy'
fct.number_densities_mass_range(
    u_all_MW, mass_range_eV, output, 
    average=False, m_start=0.01, z_start=0.5
)
etas_smooth = np.load(f'{output}')/N0

********************* Initialization *********************
# Initial conditions for neutrinos:
PHIs = 10, THETAs=10, Vs=100
Total neutrinos: 10000
# Simulation parameters:
Simulation box: L012N376
Snapshot from 0062 (z=0) to 0013 (z=4)
Pre/Sim CPUs 6/6
DM limit for cells: 1000
# File management:
Box files directory: 
 /home/fabian/ownCloud/snellius/L012N376/CDM_TF50
Output directory: 
 /home/fabian/my_github_projects/neutrino_clustering_V2/L012N376/CDM_TF50
**********************************************************
********Number density band********
Halo batch params (Rvir,Mvir,cNFW):
[[299.54209457  12.45775108   8.69090077]]
***********************************


## Plotting.

In [ ]:
fig_dir = f'{os.getcwd()}/figures'
fname = f'{SIM_TYPE}_{hname}'
fct.plot_eta_band(etas_sim, etas_smooth, mass_range_eV, fig_dir, fname, show=True)

In [ ]:
fig_dir = f'{os.getcwd()}/figures'
fname = f'{SIM_TYPE}_{hname}'
fct.plot_eta_z_back_1Halo(u_all, NU_MASSES, fig_dir, fname, show=True)


In [ ]:
fig_dir = f'{os.getcwd()}/figures'
fname = f'{SIM_TYPE}_{hname}'
fct.plot_phase_space_1Halo(
    u_all, NU_MASSES, halo_batch_params[0], 
    PRE.Vs, PRE.PHIs, PRE.THETAs, PRE.LOWER, PRE.UPPER,
    fig_dir, fname, show=True
)

In [ ]:
def plot_number_density_integral(
    sim_vels, nu_masses, halo_param_arr, 
    vels, phis, thetas, lower, upper,
    fig_dir, fname, show=False
):

    # Convert to first and last momenta (of each neutrino).
    p_arr, _ = fct.velocity_to_momentum(sim_vels, nu_masses)
    p0_arr, p1_arr = p_arr[...,0], p_arr[...,-1]

    # Sort.
    ind = p0_arr.argsort(axis=-1)
    p0_sort = np.take_along_axis(p0_arr, ind, axis=-1)
    p1_sort = np.take_along_axis(p1_arr, ind, axis=-1)

    curtain_behaviour = True

    if curtain_behaviour:
        p0_final = p0_sort
        p1_final = p1_sort
    else:
        # Each velocity has a batch of neutrinos.
        # Take median as statistic of how much clustering shifts it.
        m_len = (len(nu_masses))
        p0_blocks = p0_sort.reshape((m_len, vels, phis*thetas))
        p0_final = p0_blocks[...,0]
        p1_blocks = p1_sort.reshape((m_len, vels, phis*thetas))
        p1_final = np.median(p1_blocks, axis=-1)


    # Fermi-Dirac value with momentum at end of sim.
    FDvals = fct.Fermi_Dirac(p1_final)

    # What number density function integrates.
    y = FDvals * p0_final**2 
    x = p0_final



    fig, axs = plt.subplots(2,2, figsize=(12,12))
    fig.suptitle(
        'Inegral for number density visualized' ,
        fontsize=18
    )

    for j, m_nu in enumerate(nu_masses):

        k = j
        i = 0
        if j in (2,3):
            i = 1
            j -= 2

        # Using all velocities of sim.
        axs[i,j].set_title(f'{m_nu} eV')
        axs[i,j].semilogx(x/T_CNB, y, alpha=0.9)

        # Original Fermi-Dirac distr.
        # y_FD = fct.Fermi_Dirac(PRE.MOMENTA) * PRE.MOMENTA**2
        # x_FD = PRE.MOMENTA/T_CNB
        # axs[i,j].semilogx(x_FD, y_FD, c='orange', alpha=0.8, ls='-.')

        # Fermi-Dirac phase-space distr.
        pOG = np.geomspace(lower, upper, FDvals.shape[-1])
        FDvalsOG = fct.Fermi_Dirac(pOG) * pOG**2
        yOG = pOG/T_CNB
        axs[i,j].loglog(yOG, FDvalsOG, label='PS Fermi-Dirac', c='blue', alpha=0.7)

        # Plot settings.
        axs[i,j].set_xlabel(r'y = $p_0/T_{CNB}$')
        axs[i,j].set_ylabel(r'$FD(p_1) \cdot p_0^2$')
        axs[i,j].set_ylim(0, 6*1e-8)


    fig_out = f'{fig_dir}/integral_visuals_{fname}.pdf'
    plt.savefig(
        fig_out, facecolor=fig.get_facecolor(), edgecolor='none', 
        bbox_inches='tight'
    )
    if show:
        plt.show()
    else:
        plt.close()